## **Vidita Sports.com** Data Analysis


*   Importing Libraries
*   Creating SparkSession





In [31]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Vidita Sports.com").getOrCreate()


# **Reading File Content**

In [34]:
df = spark.read.csv("sports.csv", header=True, inferSchema=True)
df.show()

-------------------------------------------
Time: 2024-12-14 05:42:03
-------------------------------------------

+--------------------+----------+------------+--------------------+-------------------+-------------------+--------------------+------------------+
|               email| firstname|    lastname|       password_hash|      _address_city|_address_country_id|     _address_street|_address_telephone|
+--------------------+----------+------------+--------------------+-------------------+-------------------+--------------------+------------------+
|        h.e.v.s@i.ua|   Dmitriy|     Ishchuk|e77495e89138e732b...|               NULL|               NULL|                NULL|              NULL|
|h.e.v.s.i.ua@gmai...|   Dmitriy|     Ishchuk|3160f7932874e3260...|              Город|                 UA|               Улица|     093 699 3 669|
|support@vidita-sp...|   Dmitriy|     Ishchuk|8675fe6365733e0c8...|               Киев|                 UA|    ул. Дегтяревская|     093 699 3 66

> **Clean and Modify the Address Fields**





The address columns (_address_city, _address_country_id, _address_street, _address_telephone) need cleaning. Let's start by:



*   Removing leading/trailing spaces.
* Renaming columns for better readability.
* Removing any rows with missing address data.


In [61]:
# Trim spaces and rename columns
df_clean = df \
    .withColumn("city", df["_address_city"].alias("city").cast("string")) \
    .withColumn("country", df["_address_country_id"].alias("country_id").cast("string")) \
    .withColumn("street", df["_address_street"].alias("street").cast("string")) \
    .withColumn("telephone", df["_address_telephone"].alias("telephone").cast("string")) \
    .drop("_address_city", "_address_country_id", "_address_street", "_address_telephone")

# Remove rows with missing address data
df_clean = df_clean.dropna(subset=["city", "country", "street", "telephone"],  how="any")
df_clean = df_clean.drop("city", "street")
# Show the cleaned data
df_clean.show(5)


-------------------------------------------
Time: 2024-12-14 06:12:27
-------------------------------------------

+--------------------+---------+---------+--------------------+-------+-------------+
|               email|firstname| lastname|       password_hash|country|    telephone|
+--------------------+---------+---------+--------------------+-------+-------------+
|h.e.v.s.i.ua@gmai...|  Dmitriy|  Ishchuk|3160f7932874e3260...|     UA|093 699 3 669|
|support@vidita-sp...|  Dmitriy|  Ishchuk|8675fe6365733e0c8...|     UA|093 699 3 669|
|     dmis@shopium.ua|  Дмитрий|     Ищук|b4ed026834e999c63...|     UA|093 699 3 669|
|Kornienko.D77@lis...|     Дима|Корниенко|523fe6a5c2d4d9671...|     UA|    632777173|
|reklama.landing@g...|Олександр|  Баталін|c059743ae82d64133...|     UA|    963822634|
+--------------------+---------+---------+--------------------+-------+-------------+
only showing top 5 rows



# **Step 4: Data Aggregation and Analysis**

In [62]:
df_country_count = df_clean.groupBy("country").count()
df_country_count.show()

-------------------------------------------
Time: 2024-12-14 06:12:33
-------------------------------------------

+-------+-----+
|country|count|
+-------+-----+
|     UA|  738|
+-------+-----+



# **Step 5 :  Save the Data to a New CSV File**

In [63]:
df_clean.write.csv("vidita_sports_clean.csv", mode="overwrite" ,header=True)

-------------------------------------------
Time: 2024-12-14 06:12:35
-------------------------------------------

-------------------------------------------
Time: 2024-12-14 06:12:36
-------------------------------------------

